In [45]:
import os
import csv
import math
import jieba
import string
import pickle
from collections import Counter
from collections import defaultdict

# 增加字段大小限制，设置为更大的值（比如 10MB）
csv.field_size_limit(10000000)

# 读取倒排索引
def load_index(filename="index.pkl"):
    print("加载倒排索引...")
    with open(filename, 'rb') as f:
        inverted_index, doc_tfs, idf = pickle.load(f)
    print("倒排索引加载完成。")
    return inverted_index, doc_tfs, idf

# 打开并读取 HtmlFile.csv 文件
csv_filename = "HtmlContent.csv"

# 读取文件中的内容
with open(csv_filename, mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    # 跳过表头（如果有的话）
    next(csv_reader, None)
    # 将所有内容读入列表
    html_data = list(csv_reader)

# 加载倒排索引
inverted_index, doc_tfs, idf = load_index()

加载倒排索引...
倒排索引加载完成。


In [46]:
# 中文分词
def tokenize(text):
    return list(jieba.cut(text))

# 计算查询的TF-IDF向量
def compute_query_tfidf(query, idf):
    query_tokens = tokenize(query)
    query_tf = defaultdict(int)
    
    # 计算TF
    for word in query_tokens:
        query_tf[word] += 1
    total_words = len(query_tokens)
    for word in query_tf:
        query_tf[word] /= total_words
    
    # 计算TF-IDF
    query_tfidf = {}
    for word, tf in query_tf.items():
        query_tfidf[word] = tf * idf.get(word, 0)  # 默认IDF为0
    
    return query_tfidf

# 计算文档与查询的相关度（余弦相似度）
def compute_cosine_similarity(query_tfidf, doc_tfidf):
    dot_product = sum(query_tfidf.get(word, 0) * doc_tfidf.get(word, 0) for word in query_tfidf)
    query_norm = math.sqrt(sum(val**2 for val in query_tfidf.values()))
    doc_norm = math.sqrt(sum(val**2 for val in doc_tfidf.values()))
    if query_norm * doc_norm == 0:
        return 0
    return dot_product / (query_norm * doc_norm)

# 执行搜索
def search(query, inverted_index, doc_tfs, idf, top_n=10):
    print(f"\n正在搜索：{query}")
    # 计算查询的TF-IDF向量
    query_tfidf = compute_query_tfidf(query, idf)
    
    # 计算每个文档与查询的相关度
    scores = []
    for doc_id, doc_tfidf in enumerate(doc_tfs):
        score = compute_cosine_similarity(query_tfidf, doc_tfidf)
        scores.append((doc_id, score))
    
    # 按相关度排序并返回前top_n个文档
    scores.sort(key=lambda x: x[1], reverse=True)
    top_docs = scores[:top_n]
    
    return top_docs

# 定义循环次数计数器
search_count = 0

# 初始化查询日志
query_log = []

# 假设 html 文件存放在当前目录下的 HtmlFile 文件夹
html_folder = './HtmlFile/'

# 确保 HtmlFile 文件夹存在
if not os.path.exists(html_folder):
    print("HtmlFile 文件夹不存在，请检查文件路径。")
    exit()

# 定义一个用于过滤标点符号的函数
def is_valid_word(word):
    # 判断单词中是否包含特定符号或是否为特定无效单词
    invalid_symbols = ['，', '%', '‰', '、']
    invalid_words = ['与', '本', '的', '由', '多', '1', '新']
    
    # 如果单词包含无效符号或者是无效单词，则返回 False
    if any(symbol in word for symbol in invalid_symbols) or word in invalid_words:
        return False
    
    # 判断单词中是否包含标点符号
    return all(char not in string.punctuation for char in word)

# 分词并统计双词
def extract_bigrams(text):
    words = list(jieba.cut(text))  # 切割文本
    bigrams = []
    for i in range(len(words) - 1):
        # 生成双词组合（bigram）
        bigram = f"{words[i]}{words[i+1]}"
        # 过滤掉包含标点符号的双词
        if is_valid_word(words[i]) and is_valid_word(words[i+1]):
            bigrams.append(bigram)
    return bigrams

while True:
    # 根据循环次数决定搜索内容
    if search_count == 0:
        query = "南开大学ESI学科发展报告pdf"
    elif search_count == 1:
        query = "南开大学新校区"
    elif search_count == 2:
        query = "新冠肺炎防控指南漫画"
    elif search_count == 3:
        query = "exit"
    else:
        query = input("请输入查询词（输入 'exit' 退出）：")
    
    # 如果用户输入 "exit"，退出循环
    if query.lower() == 'exit':
        print("\n退出程序。")
        break
    
    # 执行搜索（这里假设search是一个已定义的搜索函数）
    top_docs = search(query, inverted_index, doc_tfs, idf, top_n=10)
    
    # 输出搜索结果
    print("搜索结果：")
    
    all_bigrams = []  # 用于收集所有搜索结果中的双词
    
    for doc_id, score in top_docs:
        # 获取网页的 标题 和 URL 和 锚文本 和 正文
        webpage_title = html_data[doc_id][0]    # 假设第一列是 标题
        webpage_url = html_data[doc_id][1]      # 假设第二列是 URL
        webpage_anchors = html_data[doc_id][2]  # 假设第三列是 锚文本
        webpage_body = html_data[doc_id][3]     # 假设第四列是 正文

        # 收集锚文本和正文中的双词
        all_bigrams.extend(extract_bigrams(webpage_anchors))
        all_bigrams.extend(extract_bigrams(webpage_body))

        # 构建本地文件路径（假设网页标题对应文件名）
        local_file_path = os.path.join(html_folder, f"{webpage_title}.html")
        
        # 检查文件是否存在
        if os.path.exists(local_file_path):
            # 替换文件路径中的空格为 %20
            local_file_path = local_file_path.replace(' ', '%20')
            
            # 构建 file:// 协议的本地路径
            local_file_url = f"file:///{os.path.abspath(local_file_path)}"
            
            print(f"网页标题: {webpage_title}, 网页链接: {webpage_url}\n本地文件链接: {local_file_url}")
        else:
            print(f"网页标题: {webpage_title}, 网页链接: {webpage_url}\n本地文件不存在")
    
    # 记录查询日志
    query_log.append(query)
    
    # 输出查询日志
    print("\n查询日志：")
    for i, log in enumerate(query_log, 1):
        print(f"{i}. {log}")
    
    # 统计双词
    bigram_counts = Counter(all_bigrams)
    common_bigrams = bigram_counts.most_common(10)
    print("\n推荐搜索：")
    
    # 输出推荐搜索短语，直接使用双词统计的前10个
    for i in range(0, len(common_bigrams), 10):
        line = "、".join([bigram for bigram, _ in common_bigrams[i:i+10]])
        print(line)
    
    # 增加搜索次数计数
    search_count += 1


正在搜索：南开大学ESI学科发展报告pdf
搜索结果：
网页标题: 南开大学ESI全球前1%学科增至15个 “微生物学”首次上榜-南开要闻-南开大学, 网页链接: http://news.nankai.edu.cn/ywsd/system/2021/03/28/030045131.shtml
本地文件链接: file:///d:\Programs\InformationRetrieval\HtmlFile\南开大学ESI全球前1%学科增至15个%20“微生物学”首次上榜-南开要闻-南开大学.html
网页标题: 南开大学“社会科学总论”学科首次进入全球前1%-南开要闻-南开大学, 网页链接: http://news.nankai.edu.cn/ywsd/system/2020/05/18/030039232.shtml
本地文件链接: file:///d:\Programs\InformationRetrieval\HtmlFile\南开大学“社会科学总论”学科首次进入全球前1%-南开要闻-南开大学.html
网页标题: 南开大学工程科学跻身ESI全球前1‰-南开要闻-南开大学, 网页链接: http://news.nankai.edu.cn/ywsd/system/2023/11/25/030059003.shtml
本地文件链接: file:///d:\Programs\InformationRetrieval\HtmlFile\南开大学工程科学跻身ESI全球前1‰-南开要闻-南开大学.html
网页标题: 南开大学环境科学与生态学跻身ESI全球前1‰-南开要闻-南开大学, 网页链接: http://news.nankai.edu.cn/ywsd/system/2024/07/17/030062633.shtml
本地文件链接: file:///d:\Programs\InformationRetrieval\HtmlFile\南开大学环境科学与生态学跻身ESI全球前1‰-南开要闻-南开大学.html
网页标题: 中央广电总台国际在线：南开大学环境科学与生态学跻身ESI全球前1‰-媒体南开-南开大学, 网页链接: http://news.nankai.edu.cn/mtnk/system/2024/07/18/030062653.shtml
本地文件链接: f